In [63]:
import folium
import osmnx as ox
import json
from sodapy import Socrata
import numpy as np
from math import log2
import random
import pandas as pd
from itertools import combinations

STOPS = '4vt2-8zrq'
TRIPS = 'ctwr-tvrd'
SCHEDULE = '4fvt-p2se'

EDMONTON_BLUE = '#035086'
COLORS=['#E21E26', #Red
        '#EE8C24', #Orange
        '#F8DB44', #Yellow
        '#107547', #Dark Green
        '#68C079', #Dark Green
        '#2D63AF', #Dark Blue
        '#5CBDDE', #Light Blue 
        '#10262F', #Black
        '#99479A', #Purple 
        '#EC9195', #Pink
        '#84999A', #Grey
        '#D9EBD ', #Muted Green
        ]

In [2]:
def random_hex_color():
    return random.choice(COLORS)

In [3]:
client = Socrata("data.edmonton.ca", None)

In [4]:
def get_stop(stop_id, client):
    return client.get(STOPS, stop_id=stop_id, limit=1)[0]

def get_stops_by_trip_id(trip_id, client):
    return client.get(SCHEDULE, trip_id=trip_id)

def get_trips_by_route_id(route_id, client):
    return client.get(TRIPS, route_id=route_id)

def get_trips_by_trip_id(trip_id, client):
    return client.get(TRIPS, trip_id=trip_id)

def get_routes_by_stop_id(stop_id, client):
    results = client.get(SCHEDULE, stop_id=stop_id)
    result_df = pd.DataFrame.from_records(results)
    routes = result_df['route_id'].unique()
    return routes 

def add_points(points, m, color=random_hex_color()):
    folium.PolyLine(locations=points, color=color).add_to(m)

In [5]:
def draw_trip(trip, client, m, color=random_hex_color()):
    points = trip['geometry_line']['coordinates'][0]
    points = [[point[1], point[0]] for point in points]
    add_points(points, m, color=color)
    #stops = get_stops_by_trip_id(trip['trip_id'], client)
    stops = []
    for stop in stops:
        s = get_stop(stop['stop_id'], client)
        pos = [s['geometry_point']['coordinates'][1],s['geometry_point']['coordinates'][0]]
        folium.Circle(location=pos, color=EDMONTON_BLUE, fill_opacity=1, fill_color=EDMONTON_BLUE, radius=4).add_to(m)

def draw_route(route_id, client, m, color=random_hex_color()):
    trip0 = client.get(TRIPS, route_id=route_id, direction_id=0, limit=1)
    #trip1 = client.get(TRIPS, route_id=route_id, direction_id=1, limit=1)

    if len(trip0):
        draw_trip(trip0[0], client, m, color=color)
    #if len(trip1):
    #    draw_trip(trip1[0], client, m, color=color)

In [52]:
def find_sublist(list1, list2):
    start = -1
    end = -1
    m = []
    for i in range(len(list1)):
        if list1[i] in list2:
            m.append(list1[i])        
            if start == -1:
                start = i
        elif start != -1 and end == -1:
            end = i
    return m, start, end  

def side_of_line(A, B, P):
    A = np.array(A)
    B = np.array(B)
    P = np.array(P)
    AP = P - A
    AB = B - A
    # Compute the cross product
    return np.cross(AP, AB)

def perpen(A, B):
    dx = B[0] - A[0]
    dy = B[1] - A[1]

    return -dy,dx

def move_point(point, d, dist):
    x, y = point
    dx, dy = d
    return x+dx*dist, y+dy*dist

def un_overlap(p1, p2):
    
    m, start, end = find_sublist(p1,p2)
    start2 = p2.index(p1[start])
    end2 = start + len(m)

    side1 = side_of_line(p1[end], p1[end-1], p1[end])
    side2 = side_of_line(p1[end], p1[end-1], p2[end2+1])

    if side1 > side2:
        dir = -1
    else:
        dir = 1

    newp1 = []    
    newp2 = []    
    dx,dy = perpen(m[0], m[len(m)-1])
    for i in range(1, len(m)):
        newp1.append(move_point(m[i], (-dx,-dy), 0.01))
        newp2.append(move_point(m[i], (dx,dy), 0.01))

    p1[start:end] = newp1
    p2[start2:end2] = newp2
    return p1, p2




routes = get_routes_by_stop_id(7143, client)
r1 = client.get(TRIPS, route_id=routes[0], limit=1)[0]['geometry_line']['coordinates'][0]
r2 = client.get(TRIPS, route_id=routes[1], limit=1)[0]['geometry_line']['coordinates'][0]


r1,r2 = un_overlap(r1, r2)

#m =  draw_stop(7143, client)

m = folium.Map()
print(r1)
print(r2)
r1 = [[point[1], point[0]] for point in r1]
r2 = [[point[1], point[0]] for point in r2]
add_points(r1, m, color=COLORS[0])
add_points(r2, m, color=COLORS[1])




m


[[-113.49006200000002, 53.59203199999787], [-113.48967900000002, 53.592029999997884], [-113.48906000000002, 53.592029999997884], [-113.48886000000003, 53.59207999999787], [-113.48783000000003, 53.59207999999787], [-113.48707500000003, 53.59208399999787], [-113.48707500000003, 53.59208399999787], [-113.48267800000004, 53.592105999997855], [-113.48267800000004, 53.592105999997855], [-113.48193000000003, 53.59210999999787], [-113.48124000000003, 53.59211999999788], [-113.48063000000003, 53.59211999999786], [-113.47971000000003, 53.59212999999786], [-113.47898000000004, 53.59213499999786], [-113.47898000000004, 53.59213499999786], [-113.47813000000002, 53.592139999997876], [-113.47619000000003, 53.59214999999786], (-113.47545296000003, 53.591830449997865), (-113.47386896000003, 53.591844449997865), (-113.47262896000002, 53.591854449997854), (-113.47085796000003, 53.591864449997864), (-113.47072896000003, 53.59183444999787), (-113.47036596000002, 53.59183644999787), (-113.47036596000002, 53

In [84]:
def draw_stop(stop_id, client):
    stop = get_stop(stop_id, client)
    stop_location = [stop['geometry_point']['coordinates'][1], stop['geometry_point']['coordinates'][0]]

    m = folium.Map(location=stop_location, zoom_start=12)

    routes = get_routes_by_stop_id(stop_id, client)
    points_list = []
    trips = []
    for route in routes:
        trip = get_trips_by_route_id(route, client)[0]
        trips.append(trip)
        points = trip['geometry_line']['coordinates'][0]
        points = [[point[1], point[0]] for point in points]
        points_list.append(points)
    
    combinations_list = combinations(range(len(points_list)), 2)

    for combination in combinations_list:
        points_list[combination[0]], points_list[combination[1]] = un_overlap(points_list[combination[0]], points_list[combination[1]] )
    
    for points in points_list:
        add_points(points, m)

    
    stops = []
    for trip in trips:
        for stop in get_stops_by_trip_id(trip['trip_id'], client):
            stops.append(stop)

    for stop in stops:
        s = get_stop(stop['stop_id'], client)
        pos = [s['geometry_point']['coordinates'][1],s['geometry_point']['coordinates'][0]]
        folium.Circle(location=pos, color=EDMONTON_BLUE, fill_opacity=1, fill_color=EDMONTON_BLUE, radius=4).add_to(m)


    folium.Marker(stop_location).add_to(m)
    return m

draw_stop(7143, client)

In [ ]:

geojson_features = []
for marker in m._children.values():
    if isinstance(marker, folium.Marker):
        feature = {
            "type": "Feature",
            "geometry": {
                "type": "Point",
                "coordinates": [marker.location[1], marker.location[0]]  # GeoJSON uses [longitude, latitude] order
           }
        }
        geojson_features.append(feature) 
    elif isinstance(marker, folium.PolyLine):
        locations = marker.locations
        locations = [[l[1], l[0]] for l in locations]
        feature = {
            "type": "Feature",
            "geometry": {
                "type": "LineString",
                "coordinates": locations 
            },
            "properties": {}
        }
        geojson_features.append(feature) 

geojson_data = {
    "type": "FeatureCollection",
    "features": geojson_features
}

with open(f"{stop_id}.geojson", "w") as geojson_file:
    json.dump(geojson_data, geojson_file)
        